# Model and dataset Folder

# External datasets created - merged, train and test

In [ ]:
# Perform an inner join to merge on 'Symbol' (this only keeps rows where there is a match)
merged_data = pd.merge(company_stock_details, company_info, on='Symbol', how='inner')

# Check the first few rows of the merged data
print(merged_data.head())

# Save the merged data into a new CSV file
merged_data.to_csv('merged_company_data.csv', index=False)

In [ ]:
# Sort by date to maintain chronological order
merged_data.sort_values(by=['Symbol', 'Date'], inplace=True)

# Define the split date
train_data = merged_data[merged_data['Date'] < '2022-01-01']
test_data = merged_data[merged_data['Date'] >= '2022-01-01']

train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

# Load libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import root_mean_squared_error

# ARIMA model

# Holt-winter method model

In [ ]:
model = ExponentialSmoothing(
    train_data,
    trend='add',
    seasonal='add',
    seasonal_periods=7
).fit()

forecast = model.forecast(steps=len(test_data))

# LSTM model

In [ ]:
model = AttentionLSTM(
    input_size=input_size,
    hidden_layer_size=hidden_layer_size,
    output_size=1,
    num_layers=num_layers
)

model.fit(
    X_train_tensor,
    y_train_tensor,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val_tensor, y_val_tensor),
    callbacks=[early_stopping, scheduler]
)

forecast = []
current_sequence = X_val[0]
with torch.no_grad():
    for i in range(len(X_val)):
        current_sequence_tensor = torch.from_numpy(current_sequence).float().unsqueeze(0)
        prediction = model(current_sequence_tensor)
        forecast.append(prediction.item())
        new_prediction = np.array([[prediction.item()] + [0] * (current_sequence.shape[1] - 1)])
        current_sequence = np.vstack((current_sequence[1:], new_prediction))

# prophet model